In [1]:
import tensorflow as tf
print("hello")


hello


In [4]:
import os
import shutil

raw_dir=r'C:\Users\USER\OneDrive\Pictures\mrlEyes_2018_01'
for dirpath, dirname, filenames in os.walk(raw_dir):
   for i  in [f for f in filenames if f.endswith('.png')]:
     if i.split('_')[4]=='0':
       shutil.copy(src=dirpath+'/'+i, dst=r'C:\Users\USER\OneDrive\Pictures\prepared_data\closed_eyes')
     elif i.split('_')[4]=='1':
      shutil.copy(src=dirpath+'/'+i, dst=r'C:\Users\USER\OneDrive\Pictures\prepared_data\open_eyes')

In [2]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,validation_split=0.2)
train_data=train_datagen.flow_from_directory(r'C:\Users\USER\OneDrive\Pictures\project\Drowsiness_detection\prepareddata\train_data',target_size=(80,80),batch_size=8,class_mode='categorical',subset='training')
validation_data=train_datagen.flow_from_directory(r'C:\Users\USER\OneDrive\Pictures\project\Drowsiness_detection\prepareddata\train_data',target_size=(80,80),batch_size=8,class_mode='categorical',
                                                 subset='validation')

Found 54100 images belonging to 2 classes.
Found 13524 images belonging to 2 classes.


In [4]:
test_datagen=ImageDataGenerator(rescale=1./255)

test_data=test_datagen.flow_from_directory(r'C:\Users\USER\OneDrive\Pictures\project\Drowsiness_detection\prepareddata\test_data',target_size=(80,80),batch_size=8,class_mode='categorical')

Found 530 images belonging to 2 classes.


In [5]:
base_model= InceptionV3(include_top=False,weights='imagenet',input_tensor=Input(shape=(80,80,3)))
head_model=base_model.output

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint=ModelCheckpoint(r'C:\Users\USER\OneDrive\Pictures\project\Drowsiness_detection\model checkpoint\model.h5',monitor='val_loss',save_best_only=True,verbose=3)

earlystop=EarlyStopping(monitor='val_loss',patience=7,verbose=3,restore_best_weights=True)
learning_rate=ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3)
callbacks=[checkpoint,earlystop,learning_rate]

In [7]:
!pip install -U scikit-learn 

In [8]:
head_model=Flatten()(head_model)
head_model=Dense(64,activation='relu')(head_model)
head_model=Dropout(0.5)(head_model)
head_model=Dense(2,activation='softmax')(head_model)

model=Model(inputs=base_model.input,outputs=head_model)

for layer in base_model.layers:
    layer.trainable=False

In [9]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [10]:
import PIL
from PIL import Image

In [13]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder
batchsize=80
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,epochs=5)

C:\Users\USER\AppData\Local\Temp\ipykernel_6192\2548614583.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5
676/676 [==============================] - ETA: 0s - loss: 0.1981 - accuracy: 0.9255
Epoch 1: val_loss improved from inf to 0.24340, saving model to C:\Users\USER\OneDrive\Pictures\project\Drowsiness_detection\model checkpoint\model.h5


C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


676/676 [==============================] - 97s 124ms/step - loss: 0.1981 - accuracy: 0.9255 - val_loss: 0.2434 - val_accuracy: 0.8898 - lr: 0.0010
Epoch 2/5
676/676 [==============================] - ETA: 0s - loss: 0.1715 - accuracy: 0.9332
Epoch 2: val_loss improved from 0.24340 to 0.22185, saving model to C:\Users\USER\OneDrive\Pictures\project\Drowsiness_detection\model checkpoint\model.h5
676/676 [==============================] - 84s 125ms/step - loss: 0.1715 - accuracy: 0.9332 - val_loss: 0.2218 - val_accuracy: 0.9053 - lr: 0.0010
Epoch 3/5
676/676 [==============================] - ETA: 0s - loss: 0.1799 - accuracy: 0.9277
Epoch 3: val_loss did not improve from 0.22185
676/676 [==============================] - 87s 129ms/step - loss: 0.1799 - accuracy: 0.9277 - val_loss: 0.2516 - val_accuracy: 0.8935 - lr: 0.0010
Epoch 4/5
676/676 [==============================] - ETA: 0s - loss: 0.1692 - accuracy: 0.9353
Epoch 4: val_loss improved from 0.22185 to 0.19732, saving model to C:\U

In [15]:
acc_tr,loss_tr=model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

6763/6763 [==============================] - 1944s 287ms/step - loss: 0.1598 - accuracy: 0.9377
0.1598142832517624
0.937689483165741


In [16]:
acc_vr,loss_vr=model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\USER\AppData\Local\Temp\ipykernel_6192\4184188003.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr,loss_vr=model.evaluate_generator(validation_data)


0.23127563297748566
0.899068295955658


In [17]:
acc_te,loss_te=model.evaluate_generator(test_data)
print(acc_te)
print(loss_te)

C:\Users\USER\AppData\Local\Temp\ipykernel_6192\3010557615.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_te,loss_te=model.evaluate_generator(test_data)


0.13687793910503387
0.9433962106704712
